In this example we are going to buid a recurrent neural network using the MNIST data set of hand written digits. This standard neural neural network are ideal to use when we need a sense of tie or sequence. The RNN keeps working weights which get updated over time.

In [1]:
import tensorflow as tf

# Check to see where the mnist gets defined

We are going to define the hyper parameters and the network parameters. After wards, we define the graph and the weights and biases.

In [ ]:
# parameters
learning_rate = 0.001
training_iters = 10000
batch_size = 128
display_step = 10

# network parameters
n_input = 28 # MNIST data input
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer number of features
n_classes = 10 # total classes

# graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# define the weights
weights = { 'out' : tf.Variable(tf.random_normal([n_hidden, n_classes])) }
biases = { 'out' : tf.Variable(tf.random_normal([n_classes])) }

Let us define the recurrent neural network. The RNN needs the parameters of x, weights and biases. We are going to prepare the data to create a basic LSTM cell. Now that we have the created LSTM cell we can get the outputs. Once we have the outputs we can now multiple the previous output of the cell with the weights of the current state and add a the bias weight. Now let's use our shinny new LSTM cell.

In [ ]:
#define the RNN
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    x = tf.transpose(x, [1, 0, 2])  # Permuting batch_size and n_steps
    x = tf.reshape(x, [-1, n_input])    # reshaping to (n_steps*batch_size, n_input)
    x = tf.split(x, n_steps, 0)       # split to get a list of n_steps tensor of shape (batch_size, n_input)
    
    # define a lstm cell w/ TF
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    # get lstm cell output
    outputs, states = lstm(lstm_cell, x)
    
    # linear activation, using rnn ineeer loop last output
    return tf.matmul(outputs[-1]. weights['out']) + biases['out']

pred = RNN(x, weights, biases)

In this section we are going to define the cost function. The cost
function in this example will reduce the mean while it does the softmax cross entropy with logits. The optimizer we are going to use is the AdamOptimizer with our learning rate we defined in our hyper parameters. We then minimize the cost by invoking the minimize function. We can now initilize all the variables.

In [ ]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learnibng_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# intialize the variables
init = tf.global_variables_initializer()

We finally have defined everything we need to start the training. We are going to start the tensorflow sesion and we are going to loop through the number of iterations we set in our hyper parameters. First, we will get the data in batches with the batch size we defined in our hyper parameter section. We will then reshape the tensor to get a 28 sequence of 28 elements. Once that is ready we can now feed it to our optimizer where it will calculate the backpropagation. In the next step, we will pring every so often to see how well the network is doing while training. We will calculate the accuracy and the loss score and print it out to see.

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    while step * batch_size < training_iters:
        # get batched training data
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # reshape the data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        
        # run opt op aka backprop
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
        
            # calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            
            print "Iter "+ str(step*batch_size) + ", Minibatch Loss = " + \
                "{:.6f}".format(loss) + ", Training Accuracy= " + \
                "{:.5f}".format(acc)
        step += 1
    print("Optimization Finished!")